In [45]:
import keras
import keras.backend as K
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import np_utils
import numpy as np
from sklearn.model_selection import train_test_split
import optuna
from datetime import datetime

In [46]:
#メインの関数を定義する
def main():
    #開始時刻
    datetime.now().strftime("%Y/%m/%d %H:%M:%S")
    
    study = optuna.create_study()
    study.optimize(objective, n_trials=100)
    
    print(study.best_params)
    print(study.best_value)
    print(study.best_trial)
    #終了時刻
    datetime.now().strftime("%Y/%m/%d %H:%M:%S")

In [47]:
num_classes = 10
img_rows, img_cols = 28, 28
image_size = 28

In [48]:
def load_data():    
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
        x_test = x_test .reshape(x_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
        x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /=255
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    return x_train,  x_test, y_train, y_test

In [49]:
def create_model(num_layer, activation, mid_units, num_filters, dropout_rate):
    """
    num_layer : 畳込み層の数
    activation : 活性化関数
    mid_units : FC層のユニット数
    num_filters : 各畳込み層のフィルタ数
    dropout_rate : 入力ユニットをドロップする割合
    """
    
    model = Sequential()
    model.add(Conv2D(num_filters[0], (3, 3), padding='same', input_shape=(image_size, image_size, 1)))
    model.add(Activation(activation))
    
    for i in range(1, num_layer):
        model.add(Conv2D(num_filters[i], (3, 3), padding='same'))
        model.add(Activation(activation))
        
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    
    model.add(Flatten())
    
    model.add(Dense(mid_units))
    model.add(Activation(activation))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    return model


In [50]:
def objective(trial):
    #セッションのクリア
    K.clear_session()
    
    #データ読み込み
    x_train,  x_test, y_train, y_test = load_data()
    
    #最適化するパラメータの設定
    #畳込み層の数
    num_layer = trial.suggest_int("num_layer", 3, 5)
    
    #FC層のユニット数
    mid_units = int(trial.suggest_discrete_uniform("mid_units", 300, 500, 10))
    
    #各畳込み層のフィルタ数
    num_filters = [int(trial.suggest_discrete_uniform("num_filter_"+str(i), 16, 128, 16)) for i in range(num_layer)]
    
    #活性化関数
    activation = trial.suggest_categorical("activation", ["relu", "sigmoid"])
    
    #optimizer
    optimizer = trial.suggest_categorical("optimizer", ["sgd", "adam", "rmsprop"])
    
    #Dropout
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 1.0)
    
    #学習率
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-7, 1e0)
    model = create_model(num_layer, activation, mid_units, num_filters, dropout_rate)
    
    if optimizer == "sgd":
        opt = keras.optimizers.SGD(learning_rate)
    elif optimizer == "adam":
        opt = keras.optimizers.adam(learning_rate)
    elif optimizer == "rmsprop":
        opt = keras.optimizers.rmsprop(learning_rate)
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt, metrics=['accuracy'])
    
    history = model.fit(x_train, y_train, 
                        batch_size=32, 
                        epochs=10)

In [ ]:
if __name__ == "__main__":
    main()

[I 2021-12-03 16:06:44,734] A new study created in memory with name: no-name-f3252394-38c9-4fb5-b0c4-5ba719d94690


Epoch 1/10
60000/60000 [==============================] - 471s 8ms/step - loss: 2.3035 - accuracy: 0.0933
Epoch 2/10
60000/60000 [==============================] - 433s 7ms/step - loss: 2.3009 - accuracy: 0.1298
Epoch 3/10
60000/60000 [==============================] - 436s 7ms/step - loss: 2.2985 - accuracy: 0.1654
Epoch 4/10
60000/60000 [==============================] - 437s 7ms/step - loss: 2.2959 - accuracy: 0.1968
Epoch 5/10
60000/60000 [==============================] - 437s 7ms/step - loss: 2.2931 - accuracy: 0.2250
Epoch 6/10
60000/60000 [==============================] - 435s 7ms/step - loss: 2.2897 - accuracy: 0.2453
Epoch 7/10
60000/60000 [==============================] - 6197s 103ms/step - loss: 2.2856 - accuracy: 0.2652
Epoch 8/10
60000/60000 [==============================] - 429s 7ms/step - loss: 2.2804 - accuracy: 0.2760
Epoch 9/10
60000/60000 [==============================] - 426s 7ms/step - loss: 2.2733 - accuracy: 0.2842
Epoch 10/10
60000/60000 [==================

[W 2021-12-03 18:55:39,428] Trial 0 failed, because the value None could not be cast to float.


Epoch 1/10
60000/60000 [==============================] - 1173s 20ms/step - loss: 0.6427 - accuracy: 0.7669
Epoch 2/10
60000/60000 [==============================] - 1180s 20ms/step - loss: 0.3820 - accuracy: 0.8623
Epoch 3/10
60000/60000 [==============================] - 1184s 20ms/step - loss: 0.3309 - accuracy: 0.8788
Epoch 4/10
60000/60000 [==============================] - 1182s 20ms/step - loss: 0.2985 - accuracy: 0.8908
Epoch 5/10
60000/60000 [==============================] - 1248s 21ms/step - loss: 0.2722 - accuracy: 0.8998
Epoch 6/10
60000/60000 [==============================] - 2622s 44ms/step - loss: 0.2516 - accuracy: 0.9064
Epoch 7/10
60000/60000 [==============================] - 1335s 22ms/step - loss: 0.2356 - accuracy: 0.9130
Epoch 8/10
60000/60000 [==============================] - 1339s 22ms/step - loss: 0.2185 - accuracy: 0.9191
Epoch 9/10
60000/60000 [==============================] - 1344s 22ms/step - loss: 0.2020 - accuracy: 0.9250
Epoch 10/10
60000/60000 [===

[W 2021-12-03 22:48:27,190] Trial 1 failed, because the value None could not be cast to float.


...

Epoch 1/10
60000/60000 [==============================] - 1302s 22ms/step - loss: 2.3072 - accuracy: 0.1127
Epoch 2/10
60000/60000 [==============================] - 1266s 21ms/step - loss: 2.3074 - accuracy: 0.1130
Epoch 3/10
60000/60000 [==============================] - 1318s 22ms/step - loss: 2.3070 - accuracy: 0.1119
Epoch 4/10
60000/60000 [==============================] - 1291s 22ms/step - loss: 2.3061 - accuracy: 0.1151
Epoch 5/10
60000/60000 [==============================] - 1245s 21ms/step - loss: 2.3063 - accuracy: 0.1150
Epoch 6/10
60000/60000 [==============================] - 1341s 22ms/step - loss: 2.3059 - accuracy: 0.1145
Epoch 7/10
60000/60000 [==============================] - 1267s 21ms/step - loss: 2.3061 - accuracy: 0.1156
Epoch 8/10
60000/60000 [==============================] - 1257s 21ms/step - loss: 2.3055 - accuracy: 0.1172
Epoch 9/10
60000/60000 [==============================] - 1315s 22ms/step - loss: 2.3055 - accuracy: 0.1175
Epoch 10/10
60000/60000 [===

[W 2021-12-06 04:50:10,527] Trial 11 failed, because the value None could not be cast to float.


Epoch 1/10
60000/60000 [==============================] - 1159s 19ms/step - loss: 191.9530 - accuracy: 0.0995
Epoch 2/10
60000/60000 [==============================] - 1170s 20ms/step - loss: 189.6615 - accuracy: 0.0994
Epoch 3/10
60000/60000 [==============================] - 1075s 18ms/step - loss: 189.2178 - accuracy: 0.1017
Epoch 4/10
60000/60000 [==============================] - 1102s 18ms/step - loss: 189.1874 - accuracy: 0.1009
Epoch 5/10
60000/60000 [==============================] - 1194s 20ms/step - loss: 188.5303 - accuracy: 0.0997
Epoch 6/10
60000/60000 [==============================] - 1152s 19ms/step - loss: 188.7309 - accuracy: 0.0985
Epoch 7/10
60000/60000 [==============================] - 1104s 18ms/step - loss: 188.8042 - accuracy: 0.0991
Epoch 8/10
60000/60000 [==============================] - 1195s 20ms/step - loss: 188.5791 - accuracy: 0.1000
Epoch 9/10
60000/60000 [==============================] - 1155s 19ms/step - loss: 189.1262 - accuracy: 0.0980
Epoch 10/1

[W 2021-12-06 07:57:43,973] Trial 12 failed, because the value None could not be cast to float.


Epoch 1/10
60000/60000 [==============================] - 2031s 34ms/step - loss: 66572392560225.1719 - accuracy: 0.1010
Epoch 2/10
60000/60000 [==============================] - 2175s 36ms/step - loss: 2.4188 - accuracy: 0.0990
Epoch 3/10
60000/60000 [==============================] - 2304s 38ms/step - loss: 2.4156 - accuracy: 0.0998
Epoch 4/10
24320/60000 [===========>..................] - ETA: 21:59 - loss: 2.4172 - accuracy: 0.0999